In [11]:
# 일부 수정해서 사용하시면 편할 겁니다.
train = pd.read_csv('https://raw.githubusercontent.com/Team-2025/DLThon/refs/heads/dev/data/train.csv')

# 지연님 생성 데이터
a = pd.read_csv('https://raw.githubusercontent.com/Team-2025/DLThon/refs/heads/dev/data/general_dialog1.csv').rename(columns={'dialogue': 'conversation'})
a['class'] = '일반 대화'
a['idx'] = range(0,len(a))
a = a[['idx','class','conversation']]

# 유찬님 생성 데이터
b = pd.read_csv('https://raw.githubusercontent.com/Team-2025/DLThon/refs/heads/dev/data/general_dialog2.csv')
b = b[b['class'] == "일반 대화"] # class에 일반대화가 아닌 conversation이 적혀있어 제거

train = pd.concat([train, a, b], axis=0,ignore_index=True).drop(columns='idx')

In [12]:
import os

save_path = "team2025"
os.makedirs(save_path, exist_ok=True)  # 폴더 없으면 생성
train.to_csv(os.path.join(save_path, "merged_train(2).csv"), index=False, encoding="utf-8-sig")

In [1]:
!pip install wordcloud
!pip install konlpy
!pip install torch
!sudo apt-get update
!sudo apt-get install -y fonts-nanum fonts-noto-cjk
!fc-list | grep -i "nanum\|noto"
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.2/539.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 105.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [konlpy]2m2/3 [konlpy]
Get:1 http://archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:2 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Get:3 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:4 http://security.ubuntu.com/ubuntu noble-security/main amd64 Packages [1643 kB]
Get:5 http://archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:6 http://archive.ubuntu.com/ubuntu noble/restricted amd64 Packages [117 kB]
Get:7 http://archive.ubuntu.com/ubuntu noble/main amd64 Packages [1808 kB]
Get:8 http://archive.ubuntu.com/ubuntu noble/multiverse amd64 Packages [331 kB]
Get:9 http://archive.ubuntu.com/ubuntu noble/universe amd64 Packages [19.3 MB

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import math
import numpy as np
import pandas as pd

import re
import konlpy
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from pathlib import Path

from typing import Optional, Dict

from sklearn.metrics import f1_score, accuracy_score
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings("ignore")

In [14]:
raw_data = pd.read_csv('team2025/merged_train(2).csv')
raw_data.head()

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [15]:
print(raw_data.columns)

Index(['class', 'conversation'], dtype='object')


In [8]:
#raw_data = raw_data.drop('Unnamed: 0', axis=1)

In [16]:
raw_data.head()

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [17]:
raw_data.groupby('class').count()

,conversation
class,
갈취 대화,981
기타 괴롭힘 대화,1094
일반 대화,1098
직장 내 괴롭힘 대화,979
협박 대화,896


In [18]:
okt = Okt()
stop_words = {"하다"}

In [19]:
def preprocess_sentence(sentence, stop_words):
    # 1. 양쪽 공백 제거
    sentence = sentence.strip()

    # 2. 특수문자 및 이모지 제거 (한글, 영어, 숫자, 기본 구두점만 허용)
    sentence = re.sub(r"[^가-힣0-9a-zA-Z.,!?~\s]", " ", sentence)

    # 3. 연속된 공백 하나로 축소 및 줄 바꿈 무시
    sentence = re.sub(r"\s+", " ", sentence)
    sentence = re.sub(r"\n", " ", sentence)

    # 4. 문장 부호 앞뒤로 공백 추가 (토큰 구분을 위함)
    sentence = re.sub(r"([?.!,~])", r" \1 ", sentence)
    sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    # 형태소 분석 (단어, 품사)
    include_tags = {"Noun", "Verb", "Adjective", "Exclamation", "Adverb"}
    pos_tags = okt.pos(sentence, stem=True, norm=True)
    # 원하는 품사만 추출
    tokens = [
        word for word, tag in pos_tags
        if tag in include_tags and len(word) > 1 and word not in stop_words
    ]
        
    return tokens

In [20]:
sample_text = raw_data['conversation'][0]
tokens = preprocess_sentence(sample_text, stop_words)
print(tokens)

['지금', '스스로', '죽이다', '달라', '애원', '아니다', '죄송하다', '혼자', '죽지', '우리', '사건', '말리', '진짜', '죽이다', '버리다', '싶다', '정말', '선택', '죽다', '가족', '죽여주다', '죄송하다', '정말', '선택', '없다', '선택', '가족', '모조리', '죽이다', '버리다', '선택', '한번', '도와주다', '그냥', '죽이다', '버리다', '이의', '없다', '제발', '도와주다']


In [21]:
raw_data['tokens'] = raw_data['conversation'].apply(lambda x: preprocess_sentence(str(x), stop_words))
raw_data.head()

,class,conversation,tokens
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,"[지금, 스스로, 죽이다, 달라, 애원, 아니다, 죄송하다, 혼자, 죽지, 우리, ..."
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,"[길동, 경찰서, 이다, 마트, 폭발물, 설치, 똑바로, 들다, 한번, 얘기, 장난..."
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,"[되게, 귀엽다, 작다, 남자, 보다, 그만하다, 놀리다, 재미없다, 지영, 이지,..."
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,"[어이, 거기, 이리, 오라, 무슨, 좋다, 보이다, 있다, 보다, 아니다, 없다,..."
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,"[저기, 혹시, 너무, 뜨겁다, 저희, 회사, 선크림, 팔다, 손등, 발라, 보다,..."


In [23]:
# ===== 1) Vocab 빌드 =====
from collections import Counter
from typing import List, Tuple, Dict, Iterable
import json

SPECIALS = ["<pad>", "<unk>", "<cls>", "<sep>"]

def build_vocab(
    token_lists: Iterable[List[str]],
    min_freq: int = 2,
    max_size: int = 30000,
    specials: List[str] = SPECIALS,
) -> Tuple[Dict[str, int], List[str], Counter]:
    """
    token_lists: 각 샘플의 토큰 리스트(iterable of list[str])
    min_freq: 최소 등장 빈도 미만 토큰은 제외
    max_size: special 포함 전체 vocab 상한 (None이면 제한 없음)
    returns: (stoi, itos, counter)
    """
    counter = Counter()
    for toks in token_lists:
        counter.update(toks)

    # 빈도 필터 + 상위 max_size-특수토큰 만큼
    most = [tok for tok, cnt in counter.most_common() if cnt >= min_freq]
    if max_size is not None:
        cap = max_size - len(specials)
        most = most[:max(0, cap)]

    itos = list(specials) + most
    stoi = {tok: i for i, tok in enumerate(itos)}
    return stoi, itos, counter

def save_vocab(path: str, itos: List[str]) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(itos, f, ensure_ascii=False)

def load_vocab(path: str) -> Tuple[Dict[str, int], List[str]]:
    with open(path, "r", encoding="utf-8") as f:
        itos = json.load(f)
    stoi = {tok: i for i, tok in enumerate(itos)}
    return stoi, itos

In [24]:
# ===== 2) 토큰 → ID 인코딩 =====
def encode_tokens(
    tokens: List[str],
    stoi: Dict[str, int],
    max_len: int = 256,
    add_cls: bool = True,
    add_sep: bool = True,
) -> Tuple[List[int], List[int]]:
    """
    tokens -> input_ids, attention_mask
    - OOV는 <unk>
    - <cls>, <sep>를 옵션으로 앞/뒤에 부착
    - max_len을 초과하면 적절히 자름
    """
    pad_id = stoi["<pad>"]
    unk_id = stoi["<unk>"]
    cls_id = stoi.get("<cls>")
    sep_id = stoi.get("<sep>")

    ids = [stoi.get(t, unk_id) for t in tokens]

    # 길이 계산 (cls/sep 포함해서 자르기)
    extra = (1 if add_cls else 0) + (1 if add_sep else 0)
    keep = max_len - extra
    keep = max(0, keep)
    ids = ids[:keep]

    if add_cls:
        ids = [cls_id] + ids
    if add_sep:
        ids = ids + [sep_id]

    attn = [1] * len(ids)
    return ids, attn

In [25]:
# ===== 3) 배치 패딩(collate) =====
import torch

def collate_batch(
    batch,
    pad_id: int,
):
    """
    batch: [{"input_ids": List[int], "attention_mask": List[int], "label": int}, ...]
    """
    bs = len(batch)
    maxlen = max(len(x["input_ids"]) for x in batch)
    input_ids = torch.full((bs, maxlen), pad_id, dtype=torch.long)
    attention_mask = torch.zeros((bs, maxlen), dtype=torch.long)
    labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)

    for i, x in enumerate(batch):
        L = len(x["input_ids"])
        input_ids[i, :L] = torch.tensor(x["input_ids"], dtype=torch.long)
        attention_mask[i, :L] = torch.tensor(x["attention_mask"], dtype=torch.long)

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [26]:
# ===== 4) 예시 파이프라인 (라벨 매핑 포함) =====
# 4-1) tokens 컬럼이 없다면 먼저 생성
# raw_data['tokens'] = raw_data['conversation'].apply(lambda s: preprocess_sentence(str(s), stop_words))

# 4-2) 라벨 매핑
labels = sorted(raw_data["class"].unique().tolist())
label2id = {
    "협박 대화": 0,
    "갈취 대화": 1,
    "직장 내 괴롭힘 대화": 2,
    "기타 괴롭힘 대화": 3,
    "일반 대화": 4,
}
id2label = {v: k for k, v in label2id.items()}

# 4-3) vocab 빌드
stoi, itos, counter = build_vocab(raw_data["tokens"], min_freq=1, max_size=20000)
pad_id = stoi["<pad>"]

# 4-4) 인코딩 (train/valid 분할은 이미 되어있다고 가정하거나 아래처럼 간단 분할)
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(raw_data, test_size=0.2, random_state=42, stratify=raw_data["class"])

def encode_row(row, max_len=256):
    ids, attn = encode_tokens(row["tokens"], stoi, max_len=max_len, add_cls=True, add_sep=True)
    return {
        "input_ids": ids,
        "attention_mask": attn,
        "label": label2id[row["class"]],
    }

train_records = [encode_row(r) for _, r in train_df.iterrows()]
valid_records = [encode_row(r) for _, r in valid_df.iterrows()]

# 4-5) PyTorch Dataset/Dataloader
from torch.utils.data import Dataset, DataLoader

class SimpleListDataset(Dataset):
    def __init__(self, records):
        self.records = records
    def __len__(self):
        return len(self.records)
    def __getitem__(self, idx):
        return self.records[idx]

train_ds = SimpleListDataset(train_records)
valid_ds = SimpleListDataset(valid_records)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,
                          collate_fn=lambda b: collate_batch(b, pad_id))
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=False,
                          collate_fn=lambda b: collate_batch(b, pad_id))

print(f"Vocab size: {len(itos)} | Labels: {label2id}")
print(next(iter(train_loader))["input_ids"].shape)  # (B, S)

Vocab size: 9563 | Labels: {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
torch.Size([16, 136])


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optional

# ----------------------------
# 1D CNN (TextCNN) Classifier
# ----------------------------
class CnnTextClassifier(nn.Module):
    """
    1D CNN (TextCNN) 기반 문서/대화 분류기.
    - input_ids: (B, S) 토큰 인덱스
    - attention_mask: (B, S) 호환성을 위해 받지만 사용하지는 않음.
                      (패딩 토큰은 0번 임베딩을 가질 것이므로)
    """
    def __init__(
        self,
        vocab_size: int,
        num_classes: int,
        emb_dim: int = 256,
        num_filters: int = 100,           # 각 커널 사이즈별 필터(채널) 수
        kernel_sizes: List[int] = [3, 4, 5], # N-gram을 흉내내는 커널 크기 리스트
        dropout: float = 0.5,
        pad_id: int = 0,
    ):
        super().__init__()

        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        
        # 병렬 1D Conv 레이어
        # 입력: (B, In_Channels=emb_dim, Length=S)
        # 출력: (B, Out_Channels=num_filters, Length_out)
        self.convs = nn.ModuleList([
            nn.Conv1d(
                in_channels=emb_dim,
                out_channels=num_filters,
                kernel_size=k
            )
            for k in kernel_sizes
        ])
        
        # 분류기
        # 각 커널에서 num_filters만큼의 피처가 나오므로, 총 len(kernel_sizes) * num_filters
        self.classifier = nn.Linear(len(kernel_sizes) * num_filters, num_classes)
        self.dropout = nn.Dropout(dropout)

        # Xavier init (선택)
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_repr: bool = False,
    ):
        # (B, S) -> (B, S, E)
        x = self.emb(input_ids)
        
        # (B, S, E) -> (B, E, S) [Conv1d는 (B, Channels, Length) 입력을 받음]
        x = x.permute(0, 2, 1)

        conv_outputs = []
        for conv in self.convs:
            conv_out = F.relu(conv(x)) # (B, num_filters, S_out)
            
            # Max-over-time pooling: 시퀀스 전체에서 가장 큰 값 하나를 뽑음
            pooled_out = F.max_pool1d(conv_out, kernel_size=conv_out.shape[2]) # (B, num_filters, 1)
            conv_outputs.append(pooled_out.squeeze(2)) # (B, num_filters)
            
        # 모든 커널의 풀링 결과를 결합
        sent_repr = torch.cat(conv_outputs, dim=1) # (B, num_filters * len(kernel_sizes))
        sent_repr = self.dropout(sent_repr)
        
        logits = self.classifier(sent_repr)

        if return_repr:
            return logits, sent_repr
        return logits

In [28]:
# ----------------------------
# 학습/평가 루프
# ----------------------------
def train_one_epoch(
    model: nn.Module,
    dataloader,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    class_weights: Optional[torch.Tensor] = None,
    grad_clip: float = 1.0,
    scheduler = None,
    use_amp: bool = True,
) -> Dict[str, float]:
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    ce = nn.CrossEntropyLoss(weight=class_weights.to(device) if class_weights is not None else None)

    losses, all_preds, all_labels = [], [], []
    for batch in tqdm(dataloader, desc="train", leave=False):
        input_ids = batch["input_ids"].to(device)
        attn = batch.get("attention_mask")
        attn = attn.to(device) if attn is not None else None
        labels = batch["labels"].to(device)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=use_amp):
            logits = model(input_ids, attention_mask=attn)
            loss = ce(logits, labels)

        scaler.scale(loss).backward()
        if grad_clip is not None:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        scaler.step(optimizer)
        scaler.update()
        if scheduler is not None:
            scheduler.step()

        losses.append(loss.item())
        all_preds += logits.argmax(dim=-1).detach().cpu().tolist()
        all_labels += labels.detach().cpu().tolist()

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="macro")
    return {"loss": sum(losses)/len(losses), "acc": acc, "f1_macro": f1}

@torch.no_grad()
def evaluate(
    model: nn.Module,
    dataloader,
    device: torch.device,
) -> Dict[str, float]:
    model.eval()
    ce = nn.CrossEntropyLoss()

    losses, all_preds, all_labels = [], [], []
    for batch in tqdm(dataloader, desc="eval", leave=False):
        input_ids = batch["input_ids"].to(device)
        attn = batch.get("attention_mask")
        attn = attn.to(device) if attn is not None else None
        labels = batch["labels"].to(device)

        logits = model(input_ids, attention_mask=attn)
        loss = ce(logits, labels)
        losses.append(loss.item())

        all_preds += logits.argmax(dim=-1).detach().cpu().tolist()
        all_labels += labels.detach().cpu().tolist()

    acc = accuracy_score(all_labels, all_preds) if all_labels else 0.0
    f1 = f1_score(all_labels, all_preds, average="macro") if all_labels else 0.0
    return {"loss": sum(losses)/len(losses), "acc": acc, "f1_macro": f1}

In [29]:
# ----------------------------
# 모델 팩토리 (1D CNN)
# ----------------------------
def create_model_cnn(
    vocab_size: int,
    num_classes: int = 5,          # 협박0, 갈취1, 직장2, 기타3, 일반4
    pad_id: int = 0,
    emb_dim: int = 256,
    num_filters: int = 100,
    kernel_sizes: List[int] = [3, 4, 5],
    dropout: float = 0.5,
) -> nn.Module:
    return CnnTextClassifier(
        vocab_size=vocab_size,
        num_classes=num_classes,
        emb_dim=emb_dim,
        num_filters=num_filters,
        kernel_sizes=kernel_sizes,
        dropout=dropout,
        pad_id=pad_id,
    )

In [30]:
# 가정: stoi, itos, train_loader, valid_loader, label2id 존재
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1D CNN 모델 생성
model = create_model_cnn(
    vocab_size=len(itos),
    num_classes=5,                # 고정 매핑(협박0, 갈취1, 직장2, 기타3, 일반4)
    pad_id=stoi["<pad>"],
    emb_dim=256,
    num_filters=100,              # 각 커널별 필터 수
    kernel_sizes=[3, 4, 5],       # 3, 4, 5-gram을 보는 CNN
    dropout=0.5,                  # CNN은 보통 드롭아웃을 0.5 정도로 비교적 높게 설정
).to(device)

# 옵티마이저 (CNN은 Transformer보다 학습률을 조금 높게 잡아도 괜찮습니다)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
scheduler = None  # 필요하면 CosineAnnealingLR 등 연결

In [32]:
# (선택) 클래스 가중치: train_df의 정수 라벨 리스트로 계산
from collections import Counter
train_labels = [rec["label"] for rec in train_records]  # 이전 단계 encode_records 기준
cnt = Counter(train_labels)
weights = torch.tensor([1.0 / max(cnt.get(i, 1), 1) for i in range(5)], dtype=torch.float)
weights = weights / weights.mean()  # 평균 1로 정규화
class_weights = weights

best_f1 = 0.0
epochs = 20
for ep in range(1, epochs+1):
    tr = train_one_epoch(model, train_loader, optimizer, device,
                         class_weights=class_weights, grad_clip=1.0, scheduler=scheduler, use_amp=True)
    va = evaluate(model, valid_loader, device)
    print(f"[{ep:02d}] train: {tr} | valid: {va}")

    if va["f1_macro"] > best_f1:
        best_f1 = va["f1_macro"]
        torch.save(model.state_dict(), "./"+str(best_f1)+"best_1dcnn_cls.pt")
        print("  ✔ saved best model (F1 ↑)")

train:   0%|          | 0/253 [00:00<?, ?it/s]

eval:   0%|          | 0/32 [00:00<?, ?it/s]

[01] train: {'loss': 0.030007338351997998, 'acc': 0.9925705794947994, 'f1_macro': 0.9924974114531327} | valid: {'loss': 1.0066186734475195, 'acc': 0.8465346534653465, 'f1_macro': 0.8435324057684817}
  ✔ saved best model (F1 ↑)


train:   0%|          | 0/253 [00:00<?, ?it/s]

eval:   0%|          | 0/32 [00:00<?, ?it/s]

...

eval:   0%|          | 0/32 [00:00<?, ?it/s]

[19] train: {'loss': 0.015149286576365481, 'acc': 0.9965329370975731, 'f1_macro': 0.9964393394390431} | valid: {'loss': 1.4891516277566552, 'acc': 0.8465346534653465, 'f1_macro': 0.8439789791879821}


train:   0%|          | 0/253 [00:00<?, ?it/s]

eval:   0%|          | 0/32 [00:00<?, ?it/s]

[20] train: {'loss': 0.009552821715416222, 'acc': 0.9980188211986132, 'f1_macro': 0.9979522424109198} | valid: {'loss': 1.6410686923190951, 'acc': 0.8465346534653465, 'f1_macro': 0.8440275629406269}
